## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [41]:
DATA_URL = "http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 60.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [3]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [42]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [6]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Некрасов Николай Алексеевич. Певица</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/n/nekrasow_n_a/>Некрасов Николай Алексеевич</a><br>\r\nПевица</h2>\r\n\r'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [43]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [8]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Некрасов Николай Алексеевич. Певица\n\n\n\nНекрасов Николай Алексеевич\r\nПевица\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Пом'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [44]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 911 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [45]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent if str.isalpha(str(pred.normal_form))] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



15/15 [==============================] - 9s 568ms/step


sentences: 100%|██████████| 911/911 [00:00<00:00, 82377.78it/s]


[['с']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [46]:
len(predictions)

911

In [47]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


11695

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [48]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [49]:
fdist = FreqDist(word.lower() for word in non_uniq_tokens)
fdist.most_common(50)[0]

('он', 798)

In [50]:
counter = 0
for (word, fr) in fdist.most_common(50):
    if word in stopwords.words("russian"):
        counter += 1
counter

29

In [51]:
(50-29)/50

0.42

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [52]:
counter = 0
for (word, fr) in fdist.most_common():
    if fr > 100:
        counter += 1
counter

13

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
